# Wheatley

> Next-generation scheduling problem solver based on GNNs and Reinforcement Learning

The authors of Wheatley created a [script for training](jssp/train.py) and a [script for solving](jssp/solve.py) the problem. Documentation is in [USAGE.md](docs/USAGE.md) For this quick showcase, I trained a model on random 15x15 instances (15 jobs, 15 machines). To solve benchmark instances, I just need to import the function implemented by Wheatley's authors. The model takes quite some time to train, and therefore the model in this showcase is trained very badly due to time constraints. Makespans will be therefore incredibly bad.

In [1]:
from pathlib import Path

import numpy as np
import torch

from generic.utils import decode_mask
from jssp.description import Description
from jssp.env.env import Env
from jssp.models.agent import Agent
from jssp.solution import Solution

# import Wheatley jssp solver
from jssp.solve import solve_instance
from jssp.utils.loaders import load_problem
from jssp.models.agent import Agent

# load trained agent
agent = Agent.load("saved_networks/agent.pkl")

# Benchmarks Wheatley can solve right now

Wheatley can solve only instances, which are the same size or smaller than those he has been trained on. Therefore I can 
I will now run Wheatley on all available JSSP benchmarks I currently have, which are 15x15 or smaller.

In [2]:
import os

def parse_instance_taillard(filename):
    '''Parses instance written in Taillard specification: http://jobshop.jjvh.nl/explanation.php
    
      Args:
        filename - file containing the instance in Taillard specification

      Returns:
        number of jobs,
        number of machines,
        the processor times for each operation,
        the order for visiting the machines
    '''

    with open(filename, 'r') as f:
        # parse number of jobs J and machines M
        J, M = map(int, f.readline().split())

        # Initialize two empty numpy arrays with dimensions J x M
        processor_times = []
        orders_of_machines = []
    
        # Read the next J lines containing processor times
        for i in range(J):
            processor_times.append(list(map(int, f.readline().split())))
    
        # Read the next J lines containing orders of machines
        for i in range(J):
            orders_of_machines.append(list(map(int, f.readline().split())))

        return J, M, processor_times, orders_of_machines

def get_all_instances_in_taillard_specification():
    '''Lists all instances in Taillard specification'''
    matching_files = []
    root_dir = "../../../benchmarks/jssp/"
    target_string = "Taillard_specification"

    for foldername, subfolders, filenames in os.walk(root_dir):
        for filename in filenames:
            filepath = os.path.join(foldername, filename)
            if target_string in filepath:
                matching_files.append(filepath)

    return matching_files

def solve_instance_taillard(instance, agent):
    '''Solves JSSP instance in Taillard specification
    
      Args:
        instance: instance to solve
        agent: agent to use for solving the problem

      Returns:
        solution
    '''
    n_j, n_m, affectations, durations = load_problem(
        instance,
        taillard_offset=True,
        deterministic=True
    )

    assert agent.env_specification.max_n_jobs >= n_j
    assert agent.env_specification.max_n_machines >= n_m
        
    solution = solve_instance(
        agent, affectations, durations, True
    )

    return solution

In [4]:
for instance in sorted(get_all_instances_in_taillard_specification()):
    J, M, _, _ = parse_instance_taillard(instance)
    if agent.env_specification.max_n_jobs < J:
        continue

    if agent.env_specification.max_n_machines < M:
        continue
        
    solution = solve_instance_taillard(instance, agent)
    print(f"Instance: {instance.split('/')[-1]}, jobs: {J}, machines: {M}, makespan: {solution.get_makespan()}")

Instance: abz5.txt, jobs: 10, machines: 10, makespan: 3749.0
Instance: abz6.txt, jobs: 10, machines: 10, makespan: 2700.0
Instance: ft06.txt, jobs: 6, machines: 6, makespan: 80.0
Instance: ft10.txt, jobs: 10, machines: 10, makespan: 2768.0
Instance: la01.txt, jobs: 10, machines: 5, makespan: 1121.0
Instance: la02.txt, jobs: 10, machines: 5, makespan: 1171.0


/Users/marosbratko/Graph-neural-networks-and-deep-reinforcement-learning-in-job-scheduling/models/Wheatley/repo/jssp/env/observation.py:127: RuntimeWarning: invalid value encountered in cast
  "edge_index": edge_index.astype("int64"),


Instance: la03.txt, jobs: 10, machines: 5, makespan: 1096.0
Instance: la04.txt, jobs: 10, machines: 5, makespan: 1296.0
Instance: la05.txt, jobs: 10, machines: 5, makespan: 839.0
Instance: la06.txt, jobs: 15, machines: 5, makespan: 1816.0
Instance: la07.txt, jobs: 15, machines: 5, makespan: 1801.0
Instance: la08.txt, jobs: 15, machines: 5, makespan: 2108.0
Instance: la09.txt, jobs: 15, machines: 5, makespan: 2001.0
Instance: la10.txt, jobs: 15, machines: 5, makespan: 2158.0
Instance: la16.txt, jobs: 10, machines: 10, makespan: 2673.0
Instance: la17.txt, jobs: 10, machines: 10, makespan: 2382.0
Instance: la18.txt, jobs: 10, machines: 10, makespan: 2602.0
Instance: la19.txt, jobs: 10, machines: 10, makespan: 2476.0
Instance: la20.txt, jobs: 10, machines: 10, makespan: 2517.0
Instance: la21.txt, jobs: 15, machines: 10, makespan: 3777.0
Instance: la22.txt, jobs: 15, machines: 10, makespan: 3843.0
Instance: la23.txt, jobs: 15, machines: 10, makespan: 3821.0
Instance: la24.txt, jobs: 15, mac

KeyboardInterrupt: 

# Dynamic JSSP

In dynamic JSSP, only a subset of jobs is known at the beginning. The rest of jobs arrives dynamically online.

The following attempt to expand L2D to being dynamic is inspired by paper [Large-scale Dynamic Scheduling for Flexible Job-shop with Random Arrivals of New Jobs by Hierarchical Reinforcement Learning](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=10114974), where authors schedule newly incoming jobs and reschedule not yet executed operations, already executed operations can not be rescheduled. During each rescheduling, they formulate static FJSP and solve it. They use cache for incoming jobs and an agent choosing either to add jobs from cache to scheduling problem, or keep them in cache. I will skip this agent and always add new jobs to scheduling problem.

Similarly to the paper, I will model the arrival of new jobs as poisson process with average arrival time following an exponential distribution.

In [5]:
from datetime import datetime

def generate_arrival_times_for_jssp(instance):
    '''Turns static JSSP instance to dynamic

      Args:
        filename of static JSSP instance

      Returns:
        list of jobs known at the beginning
        dictionary of arriving jobs as  as {time_of_arrival: (operations, machines)} 
    '''
    J, M, processor_times, orders_of_machines = parse_instance_taillard(instance)
    arrival_times = np.zeros(J, dtype=int)

    # calculate beta = 1/lambda
    flattened_processor_times = [operation_time for job in processor_times for operation_time in job]
    average_time_between_arrivals = (np.mean(flattened_processor_times) * len(processor_times[0])) / M
    
    # separate jobs into known jobs and arriving jobs
    indices = np.arange(J)
    arriving_jobs_indeces = indices[:J//2]
    np.random.shuffle(indices)
    t = 1
    for index in arriving_jobs_indeces:
        t += int(np.random.exponential(scale=average_time_between_arrivals)) + 1
        arrival_times[index] = t

    return list(arrival_times)

def save_jssp_taillard(processor_times, orders_of_machines):
    '''Saves list of jobs as static JSSP instance in taillards specification
        
      Args:
        list of jobs to save

      Returns:
        filename where JSSP instance was saved to
    '''
    assert len(processor_times) == len(orders_of_machines), "Processor times and machines do not have the same length"
    for times, machines in zip(processor_times, orders_of_machines):
        assert len(times) == len(machines), "Times and machines for a specific job do not have the same length"
        
    J = len(processor_times)
    M = max(max(machines) for machines in orders_of_machines)
    
    formatted_datetime = datetime.now().strftime("%Y%m%d%H%M%S")
    with open(f"/tmp/{J}_{M}_{formatted_datetime}.txt", 'w') as f:
        f.write(f"{J} {M}\n")
        for times in processor_times:
            f.write(" ".join(map(str, times)) + '\n')
        for orders in orders_of_machines:
            f.write(" ".join(map(str, orders)) + '\n')  

    return f"/tmp/{J}_{M}_{formatted_datetime}.txt"

According to https://github.com/jolibrain/wheatley/issues/89, I was advised to model dynamic JSSP by adding a "virtual task" for jobs not known at the beginning, which have processing time equal to the arrival time of the job. Each virtual task is performed on a separate machine.

In [6]:
def get_dynamic_jssp(instance, arrival_times):
    '''Turns static JSSP into dynamic JSSP

    Args:
      instance: static JSSP instance
      arrival_times: arrival times of jobs
    '''
    J, M, processor_times, orders_of_machines = parse_instance_taillard(instance)

    for index, arrival_time in enumerate(arrival_times):
        if arrival_time == 0:
            # job known at the start
            continue

        # add a new task with processing time equal to arrival_time
        processor_times[index].insert(0, arrival_time)

        # new task is processed on a separate machine
        M += 1
        orders_of_machines[index].insert(0, M)

    return save_jssp_taillard(processor_times, orders_of_machines)


## Dynamic experiment

In [30]:
for instance in sorted(get_all_instances_in_taillard_specification()):
    arrival_times = generate_arrival_times_for_jssp(instance)
    dynamic_instance = get_dynamic_jssp(instance, arrival_times)
    J, M, _, _ = parse_instance_taillard(dynamic_instance)
    if agent.env_specification.max_n_jobs < J:
        continue

    if agent.env_specification.max_n_machines < M:
        continue
        
    static_solution = solve_instance_taillard(instance, agent)
    print(f"Static instance: {instance.split('/')[-1]}, jobs: {J}, machines: {M}, makespan: {static_solution.get_makespan()}")
    dynamic_solution = solve_instance_taillard(dynamic_instance, agent)
    print(f"Dynamic instance: {instance.split('/')[-1]}, jobs: {J}, machines: {M}, makespan: {dynamic_solution.get_makespan()}")

    # uncomment this to print schedule correctly
    # print(f"{arrival_times=}")
    # for index, arrival_time in enumerate(arrival_times):
    #     if arrival_time > 0:    
    #         print(arrival_time, dynamic_solution.schedule[index][1:])
    #     else:
    #         print(arrival_time, dynamic_solution.schedule[index])   

Static instance: abz5.txt, jobs: 10, machines: 15, makespan: 3749.0
Dynamic instance: abz5.txt, jobs: 10, machines: 15, makespan: 3434.0
Static instance: abz6.txt, jobs: 10, machines: 15, makespan: 2700.0
Dynamic instance: abz6.txt, jobs: 10, machines: 15, makespan: 2740.0
Static instance: ft06.txt, jobs: 6, machines: 9, makespan: 80.0
Dynamic instance: ft06.txt, jobs: 6, machines: 9, makespan: 101.0
Static instance: ft10.txt, jobs: 10, machines: 15, makespan: 2768.0
Dynamic instance: ft10.txt, jobs: 10, machines: 15, makespan: 2987.0
Static instance: la01.txt, jobs: 10, machines: 10, makespan: 1121.0
Dynamic instance: la01.txt, jobs: 10, machines: 10, makespan: 1400.0
Static instance: la02.txt, jobs: 10, machines: 10, makespan: 1171.0
Dynamic instance: la02.txt, jobs: 10, machines: 10, makespan: 1238.0
Static instance: la03.txt, jobs: 10, machines: 10, makespan: 1096.0
Dynamic instance: la03.txt, jobs: 10, machines: 10, makespan: 890.0
Static instance: la04.txt, jobs: 10, machines: 10